### Задание

Создайте систему компьютерного зрения, которая будет определять тип геометрической фигуры. Используя подготовленную базу и шаблон ноутбука проведите серию экспериментов по перебору гиперпараметров нейронной сети, распознающей три категории изображений (треугольник, круг, квадрат).

1. Поменяйте количество нейронов в сети, используя следующие значения:

- один слой 10 нейронов
- один слой 100 нейронов
- один слой 5000 нейронов.

2. Поменяйте активационную функцию в скрытых слоях с `relu` на `linear`.
3. Поменяйте размеры batch_size:
- 10
- 100
- 1000

4. Выведите на экран получившиеся точности.

In [ ]:
# Подключение класса для создания нейронной сети прямого распространения
from tensorflow.keras.models import Sequential
# Подключение класса для создания полносвязного слоя
from tensorflow.keras.layers import Dense
# Подключение оптимизатора
from tensorflow.keras.optimizers import Adam
# Подключение утилит для to_categorical
from tensorflow.keras import utils
# Подключение библиотеки для загрузки изображений
from tensorflow.keras.preprocessing import image
# Подключение библиотеки для работы с массивами
import numpy as np
# Подключение библиотек для отрисовки изображений
import matplotlib.pyplot as plt
# Подключение модуля для работы с файлами
import os
# Вывод изображения в ноутбуке, а не в консоли или файле
%matplotlib inline

In [ ]:
# Загрузка датасета из облака
import gdown
gdown.download('https://storage.yandexcloud.net/aiueducation/Content/base/l3/hw_light.zip', None, quiet=True)

'hw_light.zip'

In [ ]:
# Распаковываем архив hw_light.zip в папку hw_light
!unzip -q hw_light.zip


replace hw_light/0/1.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace hw_light/0/10.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace hw_light/0/100.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace hw_light/0/101.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace hw_light/0/102.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace hw_light/0/11.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace hw_light/0/12.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace hw_light/0/13.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace hw_light/0/14.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: a
error:  invalid response [a]
replace hw_light/0/14.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: A


In [ ]:
# Путь к директории с базой
base_dir = '/content/hw_light'
# Создание пустого списка для загрузки изображений обучающей выборки
x_train = []
# Создание списка для меток классов
y_train = []
# Задание высоты и ширины загружаемых изображений
img_height = 20
img_width = 20
# Перебор папок в директории базы
for patch in os.listdir(base_dir):
    # Перебор файлов в папках
    for img in os.listdir(base_dir + '/' + patch):
        # Добавление в список изображений текущей картинки
        x_train.append(image.img_to_array(image.load_img(base_dir + '/' + patch + '/' + img,
                                                    target_size=(img_height, img_width),
                                                    color_mode='grayscale')))
        # Добавление в массив меток, соответствующих классам
        if patch == '0':
            y_train.append(0)
        elif patch == '3':
            y_train.append(1)
        else:
            y_train.append(2)

# Преобразование в numpy-массив загруженных изображений и меток классов
x_train = np.array(x_train)
y_train = np.array(y_train)
# Вывод размерностей
print('Размер массива x_train', x_train.shape)
print('Размер массива y_train', y_train.shape)

Размер массива x_train (302, 20, 20, 1)
Размер массива y_train (302,)


### **Подготовка данных**

In [ ]:
x_train = x_train.reshape(302, 400)
print(x_train.shape)
x_train = x_train.astype('float32')
x_train = x_train / 255
y_train = utils.to_categorical(y_train, 3)
print(y_train.shape)
print(y_train[12])

(302, 400)
(302, 3)
[0. 1. 0.]


### **Создание нейронной сети**

In [ ]:
results = []
neuron_list = [10, 100, 5000]
activation_list = ['relu', 'linear']
batch_list = [10, 100, 1000]
for neuron in neuron_list:
  for activation in activation_list:
    for batch in batch_list:
      model = Sequential()
      model.add(Dense(neuron, input_dim = 400, activation = activation))
      model.add(Dense(3, activation = 'softmax'))
      model.compile(loss="categorical_crossentropy", optimizer=Adam(learning_rate=0.001), metrics=["accuracy"])
      res = model.fit(x_train, y_train, batch_size=batch, epochs=10, verbose=1, shuffle=True)
      results.append(('Neurons:', neuron, 'Activation:', activation, 'Batch: ', batch,
                              'accuracy:', res.history['accuracy'][9]))

Epoch 1/10
31/31 [==============================] - 1s 4ms/step - loss: 0.9652 - accuracy: 0.5563
Epoch 2/10
31/31 [==============================] - 0s 4ms/step - loss: 0.8246 - accuracy: 0.6126
Epoch 3/10
31/31 [==============================] - 0s 3ms/step - loss: 0.7656 - accuracy: 0.7053
Epoch 4/10
31/31 [==============================] - 0s 3ms/step - loss: 0.7416 - accuracy: 0.7020
Epoch 5/10
31/31 [==============================] - 0s 3ms/step - loss: 0.7064 - accuracy: 0.7351
Epoch 6/10
31/31 [==============================] - 0s 3ms/step - loss: 0.6848 - accuracy: 0.7450
Epoch 7/10
31/31 [==============================] - 0s 3ms/step - loss: 0.6603 - accuracy: 0.7649
Epoch 8/10
31/31 [==============================] - 0s 3ms/step - loss: 0.6269 - accuracy: 0.7815
Epoch 9/10
31/31 [==============================] - 0s 3ms/step - loss: 0.6138 - accuracy: 0.7848
Epoch 10/10
31/31 [==============================] - 0s 3ms/step - loss: 0.6206 - accuracy: 0.7715
Epoch 1/10
4/4 [===

In [ ]:
for i in results:
    print(i)

('Neurons:', 10, 'Activation:', 'relu', 'Batch: ', 10, 'accuracy:', 0.7715231776237488)
('Neurons:', 10, 'Activation:', 'relu', 'Batch: ', 100, 'accuracy:', 0.3642384111881256)
('Neurons:', 10, 'Activation:', 'relu', 'Batch: ', 1000, 'accuracy:', 0.49668875336647034)
('Neurons:', 10, 'Activation:', 'linear', 'Batch: ', 10, 'accuracy:', 0.804635763168335)
('Neurons:', 10, 'Activation:', 'linear', 'Batch: ', 100, 'accuracy:', 0.6026490330696106)
('Neurons:', 10, 'Activation:', 'linear', 'Batch: ', 1000, 'accuracy:', 0.5397350788116455)
('Neurons:', 100, 'Activation:', 'relu', 'Batch: ', 10, 'accuracy:', 0.9337748289108276)
('Neurons:', 100, 'Activation:', 'relu', 'Batch: ', 100, 'accuracy:', 0.7218543291091919)
('Neurons:', 100, 'Activation:', 'relu', 'Batch: ', 1000, 'accuracy:', 0.6158940196037292)
('Neurons:', 100, 'Activation:', 'linear', 'Batch: ', 10, 'accuracy:', 0.8278145790100098)
('Neurons:', 100, 'Activation:', 'linear', 'Batch: ', 100, 'accuracy:', 0.7218543291091919)
('Neuro